# Double top pattern

## read data 

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("AUD_CAD_D_2005_202206.csv")
df.columns=['time', 'open', 'high', 'low', 'close']
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head()


,time,open,high,low,close
0,2005-01-01T22:00:00.000000000Z,0.93884,0.94460,0.93813,0.93858
1,2005-01-02T22:00:00.000000000Z,0.93874,0.94348,0.93274,0.94002
2,2005-01-03T22:00:00.000000000Z,0.94002,0.94273,0.93436,0.93646
3,2005-01-04T22:00:00.000000000Z,0.93655,0.94086,0.93297,0.93734
4,2005-01-05T22:00:00.000000000Z,0.93734,0.94246,0.93227,0.93980


## add features 

In [2]:
def pivotcandles(df1, l, n1, n2): 
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividhigh:
        return 1
    elif pividlow:
        return 3
    else:
        return 0
    
df['1st_top'] = df.apply(lambda x: pivotcandles(df, x.name,10,5), axis=1)
df['2nd_top'] = df.apply(lambda x: pivotcandles(df, x.name,5,10), axis=1)
df['bottom'] = df.apply(lambda x: pivotcandles(df, x.name,5,5), axis=1)


In [3]:
def pivotbottom(df1, l, n1, n2): 
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=0
    if(df1.low[l]>df1.low[l-n1]) and (df1.low[l]>df1.low[l+n2])and df1.bottom[l]==3:
        pividlow=1
    if pividlow ==1:
        return 1
    else:
        return 0
df['bottom2'] = df.apply(lambda x: pivotbottom(df, x.name,15,15), axis=1)


In [4]:
def firstpointpos(x):
    if x['1st_top']==1:
        return x['high']+1e-3
    else:
        return np.nan

def secondpointpos(x):

    if x['2nd_top']==1 and x['1st_top']!=1:
        return x['high']+2e-3
    else:
        return np.nan
    
def bottompointpos(x):
    if x['bottom']==3:
        return x['low']-2e-4
    else:
        return np.nan
    
df['first'] = df.apply(lambda row: firstpointpos(row), axis=1)
df['second'] = df.apply(lambda row: secondpointpos(row), axis=1)
df['lowpoint'] = df.apply(lambda row: bottompointpos(row), axis=1)


In [5]:
def bottompointpos2(x):
    if x['bottom2']==1:
        return x['low']-2e-3
    else:
        return np.nan
df['lowpoint2'] = df.apply(lambda row: bottompointpos2(row), axis=1)


## set label for data  

In [6]:
index = np.array([0]*len(df))
for i in range(len(df)):
    if df['bottom2'][i]==1:
        for a in range(len(df[df.bottom2 == 1].index.tolist())):
            for j in range(df[df.bottom2 == 1].index.tolist()[a]-14, df[df.bottom2 == 1].index.tolist()[a]+14):
                index[j] = index[j]+1
            
index=index/15
df["index"] = index

## visualization

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[500:600]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['first'], mode="markers",
                marker=dict(size=5, color="blue"),
                name="first top")
fig.add_scatter(x=dfpl.index, y=dfpl['second'], mode="markers",
                marker=dict(size=5, color="red"),
                name="second top")
fig.add_scatter(x=dfpl.index, y=dfpl['lowpoint2'], mode="markers",
                marker=dict(size=5, color="yellow"),
                name="bottom point")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()


# double bottom pattern

## read data 

In [13]:
import pandas as pd
import numpy as np
df2 = pd.read_csv("AUD_CAD_D_2005_202206.csv")
df2.columns=['time', 'open', 'high', 'low', 'close']
df2.reset_index(drop=True, inplace=True)
df2.isna().sum()
df2.head()

,time,open,high,low,close
0,2005-01-01T22:00:00.000000000Z,0.93884,0.94460,0.93813,0.93858
1,2005-01-02T22:00:00.000000000Z,0.93874,0.94348,0.93274,0.94002
2,2005-01-03T22:00:00.000000000Z,0.94002,0.94273,0.93436,0.93646
3,2005-01-04T22:00:00.000000000Z,0.93655,0.94086,0.93297,0.93734
4,2005-01-05T22:00:00.000000000Z,0.93734,0.94246,0.93227,0.93980


## add features 

In [14]:
def pivotcandles(df1, l, n1, n2): 
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 2
    elif pividhigh:
        return 1
    else:
        return 0
    
df2['1st_bottom'] = df2.apply(lambda x: pivotcandles(df2, x.name,10,5), axis=1)
df2['2nd_bottom'] = df2.apply(lambda x: pivotcandles(df2, x.name,5,10), axis=1)
df2['top'] = df2.apply(lambda x: pivotcandles(df2, x.name,5,5), axis=1)

In [15]:
def pivottop(df1, l, n1, n2): 
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividhigh=0
    if(df1.high[l]<df1.high[l-n1]) and (df1.high[l]<df1.high[l+n2])and df1.top[l]==1:
        pividhigh=1
    if pividhigh ==1:
        return 1
    else:
        return 0
df2['top2'] = df2.apply(lambda x: pivottop(df2, x.name,15,15), axis=1)

In [16]:
def first_pointpos(x):
    if x['1st_bottom']==2:
        return x['low']-1e-3
    else:
        return np.nan

def second_pointpos(x):
    if x['2nd_bottom']==2 and x['1st_bottom']!=2:
        return x['low']-2e-3
    else:
        return np.nan
    
def toppointpos(x):
    if x['top']==2:
        return x['high']+2e-4
    else:
        return np.nan
    
df2['first'] = df2.apply(lambda row: first_pointpos(row), axis=1)
df2['second'] = df2.apply(lambda row: second_pointpos(row), axis=1)
df2['highpoint'] = df2.apply(lambda row: toppointpos(row), axis=1)

In [17]:
def toppointpos2(x):
    if x['top2']==1:
        return x['high']+2e-4
    else:
        return np.nan
df2['highpoint2'] = df2.apply(lambda row: toppointpos2(row), axis=1)

## set label for data 

In [18]:
index = np.array([0]*len(df2))
for i in range(len(df2)):
    if df2['top'][i]==1:
        for a in range(len(df2[df2.top2 == 1].index.tolist())):
            for j in range(df2[df2.top2 == 1].index.tolist()[a]-14, df2[df2.top2 == 1].index.tolist()[a]+14):
                index[j] = index[j]+1
            
index=index/len(df2[df2.top2 == 1].index.tolist())
df2["index"] = index

## visualization 

In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df2[2250:2350]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['first'], mode="markers",
                marker=dict(size=5, color="blue"),
                name="first bottom")
fig.add_scatter(x=dfpl.index, y=dfpl['second'], mode="markers",
                marker=dict(size=5, color="red"),
                name="second bottom")
fig.add_scatter(x=dfpl.index, y=dfpl['highpoint2'], mode="markers",
                marker=dict(size=5, color="yellow"),
                name="top point")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()